In [ ]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

#Find root directory
root_dir = Path().absolute()
if root_dir.parts[-2:] == ('notebooks', 'algae_bloom'):
    root_dir = Path(*root_dir.parts[:-2])
root_dir = str(root_dir)

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env") 

/Users/isabell/Documents/KTH/KTH femman/Scalable Machine learning/id2223-lab1/.venv/bin/python
Local environment
Root dir: /Users/isabell/Documents/KTH/KTH femman/Scalable Machine learning/id2223-lab1
Added the following directory to the PYTHONPATH: /Users/isabell/Documents/KTH/KTH femman/Scalable Machine learning/id2223-lab1
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Feature Backfill for Water Temperature Data</span>



## 🌐 Imports

In [3]:
import datetime
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

## Hopsworks connection


In [4]:
project = hopsworks.login(engine="python")

2025-11-18 18:27:19,553 INFO: Initializing external client
2025-11-18 18:27:19,554 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 18:27:21,359 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1272015


## 👉 Setting variables

In [ ]:
today = datetime.date.today() 

backfill_file = f"{root_dir}/data/water_temp_data.csv" #<-- ÄNDRA HÄR
util.check_file_path(backfill_file)

aqicn_bath_locations = settings.batch_locations #<-- ÄNDRA HÄR
coordinates = settings.coordinates #<-- ÄNDRA HÄR

Found AQICN_API_KEY: 457af4ea34575c00a7d57e82143c0860cd1c78c4
Replacing existing AQICN_API_KEY
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

## 🖼️ Create DataFrame for Water Temperature

The cell below will read up historical air quality data as a CSV file into a Pandas DataFrame

In [ ]:
#Store all water temperature measures for all bath locations in dataframe
raw_data_df = pd.read_csv(backfill_file,  parse_dates=['date'], skipinitialspace=True)

raw_data_df

File successfully found at the path: /Users/isabell/Documents/KTH/KTH femman/Scalable Machine learning/id2223-lab1/data/Edinburgh sensors/edinburgh-tower street-air-quality.csv
File successfully found at the path: /Users/isabell/Documents/KTH/KTH femman/Scalable Machine learning/id2223-lab1/data/Edinburgh sensors/edinburgh-salamander st-air-quality.csv
File successfully found at the path: /Users/isabell/Documents/KTH/KTH femman/Scalable Machine learning/id2223-lab1/data/Edinburgh sensors/edinburgh-st leonards-air-quality.csv
File successfully found at the path: /Users/isabell/Documents/KTH/KTH femman/Scalable Machine learning/id2223-lab1/data/Edinburgh sensors/edinburgh-balmwell terrace-air-quality.csv
File successfully found at the path: /Users/isabell/Documents/KTH/KTH femman/Scalable Machine learning/id2223-lab1/data/Edinburgh sensors/edinburgh-st john's road-air-quality.csv
File successfully found at the path: /Users/isabell/Documents/KTH/KTH femman/Scalable Machine learning/id2223

,date,pm25,pm10,o3,no2,so2,co,country,city,street,url,min,max,q1,q3,stdev,count
0,2025-11-01,16.0,8.0,NaN,4.0,NaN,NaN,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-11-02,17.0,4.0,NaN,4.0,NaN,NaN,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-11-03,5.0,9.0,NaN,11.0,NaN,NaN,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-11-04,15.0,11.0,NaN,10.0,NaN,NaN,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-11-05,20.0,20.0,NaN,16.0,NaN,NaN,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20567,2014-12-31,NaN,NaN,NaN,3.0,NaN,NaN,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/,NaN,NaN,NaN,NaN,NaN,NaN
20568,2015-02-26,NaN,NaN,NaN,8.0,NaN,NaN,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/,NaN,NaN,NaN,NaN,NaN,NaN
20569,2014-10-11,NaN,NaN,NaN,10.0,NaN,NaN,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/,NaN,NaN,NaN,NaN,NaN,NaN
20570,2014-12-08,NaN,NaN,NaN,18.0,NaN,NaN,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/,NaN,NaN,NaN,NaN,NaN,NaN


## 🧹Clean data

In [ ]:
#Select columns
water_temp_df = raw_data_df[['date', 'water_temp', 'coordinates']] #<-- ÄNDRA

water_temp_df

,date,pm25,country,city,street,url
0,2025-11-01,16.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
1,2025-11-02,17.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
2,2025-11-03,5.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
3,2025-11-04,15.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
4,2025-11-05,20.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
...,...,...,...,...,...,...
20567,2014-12-31,NaN,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/
20568,2015-02-26,NaN,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/
20569,2014-10-11,NaN,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/
20570,2014-12-08,NaN,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/


## 💧 Drop missing columns

In [ ]:
water_temp_df.dropna(inplace=True)
water_temp_df

,date,pm25,country,city,street,url
0,2025-11-01,16.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
1,2025-11-02,17.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
2,2025-11-03,5.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
3,2025-11-04,15.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
4,2025-11-05,20.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
...,...,...,...,...,...,...
19318,2018-06-26,36.0,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/
19319,2018-06-27,43.0,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/
19320,2018-06-28,46.0,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/
19321,2018-06-29,35.0,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/


## 🧺Add Values to Dataframe

In [ ]:
#Add values to dataframe

water_temp_df['country']= xx #<-- KOLLA OM DET BEHÖVS

water_temp_df

,date,pm25,country,city,street,url
0,2025-11-01,16.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
1,2025-11-02,17.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
2,2025-11-03,5.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
3,2025-11-04,15.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
4,2025-11-05,20.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
...,...,...,...,...,...,...
19318,2018-06-26,36.0,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/
19319,2018-06-27,43.0,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/
19320,2018-06-28,46.0,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/
19321,2018-06-29,35.0,united-kingdom,edinburgh,queensferry road,https://api.waqi.info/feed/@5985/


In [ ]:
water_temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17058 entries, 0 to 19322
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     17058 non-null  datetime64[ns]
 1   pm25     17058 non-null  float32       
 2   country  17058 non-null  object        
 3   city     17058 non-null  object        
 4   street   17058 non-null  object        
 5   url      17058 non-null  object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 866.2+ KB


## ⏮️ Calculate and Add Legged Values

In [ ]:
#Sort to make sure lagged values are correct

water_temp_df = water_temp_df.sort_values(["batch_location", "date"]) # First sort by batch location, then date <-- KOLLA OM NAMN SKA ÄNDRAS

water_temp_df

,date,pm25,country,city,street,url
10546,2020-01-11,0.45,united-kingdom,edinburgh,balmwell terrace,https://api.waqi.info/feed/A93199/
10547,2020-01-12,0.90,united-kingdom,edinburgh,balmwell terrace,https://api.waqi.info/feed/A93199/
10548,2020-01-13,2.86,united-kingdom,edinburgh,balmwell terrace,https://api.waqi.info/feed/A93199/
10549,2020-01-14,1.88,united-kingdom,edinburgh,balmwell terrace,https://api.waqi.info/feed/A93199/
10550,2020-01-15,1.45,united-kingdom,edinburgh,balmwell terrace,https://api.waqi.info/feed/A93199/
...,...,...,...,...,...,...
8,2025-11-09,27.00,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
9,2025-11-10,20.00,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
10,2025-11-11,9.00,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/
11,2025-11-12,14.00,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/


In [ ]:
#Adding columns for lagged air quality

#Takes which sensor it is into account when running A
water_temp_df["lagged_wt_1_day"]  = water_temp_df.groupby("bath_location")["pm25"].shift(1) #<-- KOLLA VARIABLESNAMN + ev felhantera
water_temp_df["lagged_wt_2_days"] = water_temp_df.groupby("bath_location")["pm25"].shift(2)
water_temp_df["lagged_wt_3_days"] = water_temp_df.groupby("bath_location")["pm25"].shift(3)

water_temp_df.dropna(inplace=True)
water_temp_df

,date,pm25,country,city,street,url,lagged_aq_1_day,lagged_aq_2_days,lagged_aq_3_days
10549,2020-01-14,1.88,united-kingdom,edinburgh,balmwell terrace,https://api.waqi.info/feed/A93199/,2.86,0.90,0.45
10550,2020-01-15,1.45,united-kingdom,edinburgh,balmwell terrace,https://api.waqi.info/feed/A93199/,1.88,2.86,0.90
10551,2020-01-16,4.11,united-kingdom,edinburgh,balmwell terrace,https://api.waqi.info/feed/A93199/,1.45,1.88,2.86
10552,2020-01-17,1.66,united-kingdom,edinburgh,balmwell terrace,https://api.waqi.info/feed/A93199/,4.11,1.45,1.88
10553,2020-01-18,0.96,united-kingdom,edinburgh,balmwell terrace,https://api.waqi.info/feed/A93199/,1.66,4.11,1.45
...,...,...,...,...,...,...,...,...,...
8,2025-11-09,27.00,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,43.00,38.00,46.00
9,2025-11-10,20.00,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,27.00,43.00,38.00
10,2025-11-11,9.00,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,20.00,27.00,43.00
11,2025-11-12,14.00,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,9.00,20.00,27.00


In [ ]:
#Double check
water_temp_df[water_temp_df["bath_location"] == "XXX"] #<-- FYLL I

,date,pm25,country,city,street,url,lagged_aq_1_day,lagged_aq_2_days,lagged_aq_3_days
2372,2019-03-21,14.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,8.0,11.0,11.0
2373,2019-03-22,22.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,14.0,8.0,11.0
2374,2019-03-23,23.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,22.0,14.0,8.0
2375,2019-03-24,18.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,23.0,22.0,14.0
2376,2019-03-25,21.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,18.0,23.0,22.0
...,...,...,...,...,...,...,...,...,...
8,2025-11-09,27.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,43.0,38.0,46.0
9,2025-11-10,20.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,27.0,43.0,38.0
10,2025-11-11,9.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,20.0,27.0,43.0
11,2025-11-12,14.0,united-kingdom,edinburgh,tower street,https://api.waqi.info/feed/@11656/,9.0,20.0,27.0


---

<span style="font-width:bold; font-size: 3rem; color:#333;">- Feature Backfill for Weather Data</span>

## 🌦 Loading Weather Data from [Open Meteo](https://open-meteo.com/en/docs)

## <span style='color:#ff5f27'> 🌍 STEP 9: Download the Historical Weather Data </span>

https://open-meteo.com/en/docs/historical-weather-api#hourly=&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant

We will download the historical weather data for your `city` by first extracting the earliest date from your DataFrame containing the historical air quality measurements.

We will download all daily historical weather data measurements for your `city` from the earliest date in your air quality measurement DataFrame. It doesn't matter if there are missing days of air quality measurements. We can store all of the daily weather measurements, and when we build our training dataset, we will join up the air quality measurements for a given day to its weather features for that day. 

The weather features we will download are:

 * `temperature (average over the day)`
 * `precipitation (the total over the day)`
 * `wind speed (average over the day)`
 * `wind direction (the most dominant direction over the day)`


In [ ]:
earliest_wt_date = pd.Series.min(water_temp_df['date'])
earliest_wt_date = earliest_wt_date.strftime('%Y-%m-%d')
earliest_wt_date

#Store all unique measure times
measure_times_df = water_temp_df[["date", "batch_location"]].copy()

weather_df = None

#Get weather for each location coordinates
for bath_dict in location_array: #<-- SE TILL ATT DETTA FUNGERAR MED VÅR LOGIK
        #Extract variables
        batch_location = bath_dict["batch_location"]
        latitude = bath_dict["latitude"]
        longitude = bath_dict["longitude"]

        #Extract target times used for retrieving weather
        spec_measure_times_df = measure_times_df[measure_times_df["batch_location"] == batch_location]

        #Get weather for each measurement for this location
        weather_df_temp = util.get_historical_weather(spec_measure_times_df, earliest_wt_date, str(today), latitude, longitude)
        
        #Update values in dataframe
        weather_df_temp["latitude"] = latitude
        weather_df_temp["longitude"] = longitude
        weather_df_temp["batch_location"] = batch_location

        #Merge with total weather datafram
        if weather_df is None:
            weather_df =  weather_df_temp
        else:
            weather_df = pd.concat([weather_df, weather_df_temp], ignore_index=True)


Coordinates 55.92266845703125°N -3.128814697265625°E
Elevation 55.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [17]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4337 entries, 0 to 4336
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         4337 non-null   datetime64[ns]
 1   temperature_2m_mean          4337 non-null   float32       
 2   precipitation_sum            4337 non-null   float32       
 3   wind_speed_10m_max           4337 non-null   float32       
 4   wind_direction_10m_dominant  4337 non-null   float32       
 5   city                         4337 non-null   object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 135.7+ KB


## 📋 Define validation rules

In [ ]:
#For water temperature data

import great_expectations as ge
wt_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="wt_expectation_suite"
)

wt_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"water_temperature",
            "min_value":-20,
            "max_value":50,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

In [ ]:
#For weather data

import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

---

## 🤫 Connect to Hopsworks and save secrets

In [20]:
fs = project.get_feature_store() 

#### Save country, city, street names as a secret

These will be downloaded from Hopsworks later in the (1) daily feature pipeline and (2) the daily batch inference pipeline

In [ ]:
#Store the name of the bath location and coordinates as secrets in Hopsworks

dict_array = []

for latitude, longitude, bath_location in wt_dict.items():
    dict_obj = {
        "bath_location": bath_location,
        "latitude": latitude,
        "longitude": longitude
    }

    dict_array.append(dict_obj)
    print(dict_array)

#Convert to json
secret_value = json.dumps(dict_array)

#If secret already exists, we replace
secrets = hopsworks.get_secrets_api()

secret = secrets.get_secret("SENSOR_LOCATIONS_JSON")
if secret is not None:
    secret.delete()
    print("Replacing existing SENSOR_LOCATIONS_JSON")


secrets.create_secret("SENSOR_LOCATIONS_JSON", secret_value)

[{'country': 'united-kingdom', 'city': 'edinburgh', 'street': 'tower street', 'aqicn_url': 'https://api.waqi.info/feed/@11656/', 'latitude': '55.956258', 'longitude': '-3.191111'}]
[{'country': 'united-kingdom', 'city': 'edinburgh', 'street': 'tower street', 'aqicn_url': 'https://api.waqi.info/feed/@11656/', 'latitude': '55.956258', 'longitude': '-3.191111'}, {'country': 'united-kingdom', 'city': 'edinburgh', 'street': 'salamander st', 'aqicn_url': 'https://api.waqi.info/feed/@5986/', 'latitude': '55.956258', 'longitude': '-3.191111'}]
[{'country': 'united-kingdom', 'city': 'edinburgh', 'street': 'tower street', 'aqicn_url': 'https://api.waqi.info/feed/@11656/', 'latitude': '55.956258', 'longitude': '-3.191111'}, {'country': 'united-kingdom', 'city': 'edinburgh', 'street': 'salamander st', 'aqicn_url': 'https://api.waqi.info/feed/@5986/', 'latitude': '55.956258', 'longitude': '-3.191111'}, {'country': 'united-kingdom', 'city': 'edinburgh', 'street': 'st leonards', 'aqicn_url': 'https:/

## 🎨 Create feature groups

In [ ]:
current_version = 1

water_temperature_fg = fs.get_or_create_feature_group(
    name='water_temperature',
    description='Water temperature at batch locations in Södertälje each day',
    version=current_version,
    primary_key=['country','city', 'street'],
    event_time="date",
    expectation_suite=wt_expectation_suite
)

In [ ]:
#Insert water temperature dataframe into feature group
water_temperature_fg.insert(water_temp_df)

2025-11-18 18:27:34,208 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272015/fs/1258614/fg/1703474


Uploading Dataframe: 100.00% |██████████| Rows 17040/17040 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: air_quality_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272015/jobs/named/air_quality_3_offline_fg_materialization/executions


(Job('air_quality_3_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747713
         }
       },
       "result": {
         "observed_value": 0.10000000149011612,
         "element_count": 17040,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T05:27:34.000207Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "

In [ ]:
#Descriptions of features

water_temperature_fg.update_feature_description("date", "Date snf time of measurement of water temperature")
water_temperature_fg.update_feature_description("bath_location", "Name of the bath location where the water temperature was measured")
water_temperature_fg.update_feature_description("latitude", "Latitude of sensor measuring water temperature")
water_temperature_fg.update_feature_description("longitude", "Longitude of sensor measuring water temperature")
water_temperature_fg.update_feature_description("lagged_wt_1_day", "Water temperature measured yesterday")
water_temperature_fg.update_feature_description("lagged_wt_2_days", "Water temperature measured two days ago")
water_temperature_fg.update_feature_description("lagged_wt_3_days", "Water temperature measured three days ago")

In [ ]:

#Create feature group for whether data

w_version = 1

weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day at point in time corresponding to measurement',
    version=w_version,
    primary_key=['bath_location'],
    event_time="date",
    expectation_suite=weather_expectation_suite
)  

#### Insert the DataFrame into the Feature Group

In [ ]:
#Insert wather dataframe into feature group
weather_fg.insert(weather_df, wait=True)

2025-11-18 18:27:56,882 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272015/fs/1258614/fg/1703478


Uploading Dataframe: 100.00% |██████████| Rows 4337/4337 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272015/jobs/named/weather_2_offline_fg_materialization/executions
2025-11-18 18:28:15,582 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 18:28:18,891 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 18:30:29,041 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 18:30:29,240 INFO: Waiting for log aggregation to finish.
2025-11-18 18:30:37,874 INFO: Execution finished successfully.


(Job('weather_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747717
         }
       },
       "result": {
         "observed_value": 3.3190360069274902,
         "element_count": 4337,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T05:27:56.000881Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_typ

In [ ]:
#Description of features in feature group

weather_fg.update_feature_description("date", "Date and time of measurement of weather")
weather_fg.update_feature_description("bath_location", "Name of bath_location where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")

---